In [ ]:
'''
Homework 8 - wafer defect
Name: Austin Ramsey
ID: 000749074
LTU Honor Code: “I have neither given nor received any unauthorized aid in completing this work, nor have I presented someone else's work as my own.”

'''

"\nHomework 8 - wafer defect\nName: Austin Ramsey\nID: 000749074\nLTU Honor Code: “I have neither given nor received any unauthorized aid in completing this work, nor have I presented someone else's work as my own.”\n\n"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Dropout, MaxPooling2D, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, VGG19, InceptionV3
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from scipy.stats import mode

2023-05-02 14:11:54.302425: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
# Directories
#train_dir = '/content/drive/MyDrive/DL_data/wafer_defect/train'
#validation_dir = '/content/drive/MyDrive/DL_data/wafer_defect/validation'
#test_dir = '/content/drive/MyDrive/DL_data/wafer_defect/test'
train_dir = '/home/robofest/Downloads/wafer_defect/train'
validation_dir = '/home/robofest/Downloads/wafer_defect/validation'
test_dir = '/home/robofest/Downloads/wafer_defect/test'


In [ ]:
# Image Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2
)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=200,
    class_mode='categorical',
    shuffle=False)

Found 8224 images belonging to 9 classes.
Found 2115 images belonging to 9 classes.
Found 2116 images belonging to 9 classes.


In [ ]:
'''
# Model 1 using VGG16 as feature extractor
conv_base1 = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(299, 299, 3))

# Model 2 using InceptionV3 as feature extractor
conv_base2 = InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(299, 299, 3))

# Model 1
model1 = Sequential()
model1.add(conv_base1)
model1.add(GlobalAveragePooling2D())
model1.add(Dense(1024, activation='relu'))
model1.add(Dense(9, activation='softmax'))

# Model 2
model2 = Sequential()
model2.add(conv_base2)
model2.add(GlobalAveragePooling2D())
model2.add(Dense(1024, activation='relu'))
model2.add(Dense(9, activation='softmax'))
'''
# Model 3
model3 = Sequential()
model3.add(Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)))
model3.add(MaxPooling2D(2, 2))
model3.add(Conv2D(64, (3, 3), activation='relu'))
model3.add(MaxPooling2D(2, 2))
model3.add(Conv2D(128, (3, 3), activation='relu'))
model3.add(MaxPooling2D(2, 2))
model3.add(Conv2D(128, (3, 3), activation='relu'))
model3.add(MaxPooling2D(2, 2))
model3.add(Flatten())
model3.add(Dense(512, activation='relu'))
model3.add(Dense(9, activation='softmax'))

# Freeze the convolutional base (pre-trained layers)
#conv_base1.trainable = False
#conv_base2.trainable = False

# Model Compilation
#model1.compile(loss='categorical_crossentropy',
#              optimizer=Adam(learning_rate=0.001),
#              metrics=['accuracy'])
#model2.compile(loss='categorical_crossentropy',
#              optimizer=Adam(learning_rate=0.001),
#              metrics=['accuracy'])
model3.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.001),
              metrics=['accuracy'])

# Models Summary
#model1.summary()
#model2.summary()
model3.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 297, 297, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 148, 148, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 128)      0

In [ ]:
# Callbacks list
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#model_checkpoint_vgg16 = ModelCheckpoint('/home/robofest/Downloads/wafer_defect/VGG16_Model.h5', monitor='val_loss', save_best_only=True)
#model_checkpoint_inceptionv3 = ModelCheckpoint('/home/robofest/Downloads/wafer_defect/InceptionV3_Model.h5', monitor='val_loss', save_best_only=True)
model_checkpoint_custom = ModelCheckpoint('/home/robofest/Downloads/wafer_defect/Custom_Model.h5', monitor='val_loss', save_best_only=True)
reduce_lr = reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_delta=1e-4, min_lr=1e-6)

#callback_list_vgg16 = [early_stopping, model_checkpoint_vgg16, reduce_lr]
#callback_list_inceptionv3 = [early_stopping, model_checkpoint_inceptionv3, reduce_lr]
callback_list_custom = [early_stopping, model_checkpoint_custom, reduce_lr]


# Train Models
print('************************')
print('*** INITIAL TRAINING ***')
print('************************\n')

# Train Custom CNN Model
print('*** Custom CNN Start ***')
history3 = model3.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    callbacks=callback_list_custom,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)
'''
# Train VGG16 Model
print('*** VGG16 Start ***')
history1 = model1.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    callbacks=callback_list_vgg16,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)
print('*** VGG16 End ***\n')

# Train InceptionV3 Model
print('*** InceptionV3 Start ***')
history2 = model2.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    callbacks=callback_list_inceptionv3,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)
print('*** InceptionV3 End ***\n')

# Load best weights from initial training
model1.load_weights('/home/robofest/Downloads/wafer_defect/VGG16_Model.h5')
model2.load_weights('/home/robofest/Downloads/wafer_defect/InceptionV3_Model.h5')

# Unfreeze layers of pre-trained models
for layer in model1.layers[:249]:
    layer.trainable = False
for layer in model1.layers[249:]:
    layer.trainable = True

for layer in model2.layers[:249]:
    layer.trainable = False
for layer in model2.layers[249:]:
    layer.trainable = True

# Compile models
model1.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])
model2.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])

# Fine-tuning VGG16 Model
print('*** VGG16 Fine-tuning Start ***')
history1b = model1.fit(
  train_generator,
  steps_per_epoch=train_generator.samples // train_generator.batch_size,
  epochs=20,
  callbacks=callback_list_vgg16,
  validation_data=validation_generator,
  validation_steps=validation_generator.samples // validation_generator.batch_size
)
print('*** VGG16 Fine-tuning End ***\n')

# Fine-tuning InceptionV3 Model
print('*** InceptionV3 Fine-tuning Start ***')
history2b = model2.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    callbacks=callback_list_inceptionv3,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)
print('*** InceptionV3 Fine-tuning End ***\n')

print('****************************')
print('*** FINE-TUNING COMPLETE ***')
print('****************************')
'''

************************
*** INITIAL TRAINING ***
************************

*** Custom CNN Start ***
Epoch 1/10
257/257 [==============================] - 628s 2s/step - loss: 1.2259 - accuracy: 0.5563 - val_loss: 0.8320 - val_accuracy: 0.7074 - lr: 0.0010
Epoch 2/10
257/257 [==============================] - 615s 2s/step - loss: 0.7904 - accuracy: 0.7099 - val_loss: 0.5533 - val_accuracy: 0.8016 - lr: 0.0010
Epoch 3/10
257/257 [==============================] - 613s 2s/step - loss: 0.6854 - accuracy: 0.7513 - val_loss: 0.5136 - val_accuracy: 0.8215 - lr: 0.0010
Epoch 4/10
257/257 [==============================] - 612s 2s/step - loss: 0.6075 - accuracy: 0.7820 - val_loss: 0.5706 - val_accuracy: 0.7921 - lr: 0.0010
Epoch 5/10
257/257 [==============================] - 614s 2s/step - loss: 0.5625 - accuracy: 0.8003 - val_loss: 0.4464 - val_accuracy: 0.8518 - lr: 0.0010
Epoch 6/10
257/257 [==============================] - 611s 2s/step - loss: 0.5384 - accuracy: 0.8104 - val_loss: 0.5081

"\n# Train VGG16 Model\nprint('*** VGG16 Start ***')\nhistory1 = model1.fit(\n    train_generator,\n    steps_per_epoch=train_generator.samples // train_generator.batch_size,\n    epochs=10,\n    callbacks=callback_list_vgg16,\n    validation_data=validation_generator,\n    validation_steps=validation_generator.samples // validation_generator.batch_size\n)\nprint('*** VGG16 End ***\n')\n\n# Train InceptionV3 Model\nprint('*** InceptionV3 Start ***')\nhistory2 = model2.fit(\n    train_generator,\n    steps_per_epoch=train_generator.samples // train_generator.batch_size,\n    epochs=10,\n    callbacks=callback_list_inceptionv3,\n    validation_data=validation_generator,\n    validation_steps=validation_generator.samples // validation_generator.batch_size\n)\nprint('*** InceptionV3 End ***\n')\n\n# Load best weights from initial training\nmodel1.load_weights('/home/robofest/Downloads/wafer_defect/VGG16_Model.h5')\nmodel2.load_weights('/home/robofest/Downloads/wafer_defect/InceptionV3_Mode

In [ ]:
# Plot validation graphs for Custom CNN
acc = history3.history['accuracy']
val_acc = history3.history['val_accuracy']
loss = history3.history['loss']
val_loss = history3.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b-', label='Training accuracy')
plt.plot(epochs, val_acc, 'y-', label='Validation accuracy')
plt.title('Training and validation accuracy Custom CNN')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b-', label='Training loss')
plt.plot(epochs, val_loss, 'y-', label='Validation loss')
plt.title('Training and validation loss Custom CNN')
plt.legend()

plt.show()

In [ ]:
#model1.load_weights('/content/drive/MyDrive/DL_data/wafer_defect/VGG16_Model.h5')
#model2.load_weights('/content/drive/MyDrive/DL_data/wafer_defect/InceptionV3_Model.h5')
model3.load_weights('/home/robofest/Downloads/wafer_defect/Custom_Model.h5')

# Model 1 Evaluation
#test_loss1, test_acc1 = model1.evaluate(test_generator)

# Model 2 Evaluation
#test_loss2, test_acc2 = model2.evaluate(test_generator)

# Model 3 Evaluation
test_loss3, test_acc3 = model3.evaluate(test_generator)

# Show Accuracy
#mean_acc = (test_acc1+test_acc2+test_acc3)/3

print('Test Accuracy: ', test_acc3*100)

11/11 [==============================] - 24s 2s/step - loss: 0.3187 - accuracy: 0.8913
Test Accuracy:  89.13043737411499


In [ ]:
# Obtain predictions from each model
#pred1 = model1.predict(test_generator)
#pred2 = model2.predict(test_generator)
pred3 = model3.predict(test_generator)

# Convert predictions to binary class labels
#labels1 = np.round(pred1).flatten()
#labels2 = np.round(pred2).flatten()
labels3 = np.round(pred3).flatten()

# Perform majority voting
voting_matrix = np.vstack((labels3))
majority_labels, _ = mode(voting_matrix, axis=0)
majority_labels = majority_labels.flatten()

# Calculate ensemble accuracy
true_labels = test_generator.classes

correct_predictions = np.sum(majority_labels == true_labels)
total_predictions = len(true_labels)

ensemble_accuracy = correct_predictions / total_predictions

print('Ensemble test accuracy (majority voting with mode):', ensemble_accuracy)

11/11 [==============================] - 22s 2s/step
Ensemble test accuracy (majority voting with mode): 0.19848771266540643


/tmp/ipykernel_107542/2948801362.py:13: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  majority_labels, _ = mode(voting_matrix, axis=0)


In [ ]:
# Summarize the results

#print(f"The best model 1 test accuracy obtained: {test_acc1*100}%")
#print(f"The best model 2 test accuracy obtained: {test_acc2*100}%")
print(f"The best model 3 test accuracy obtained: {test_acc3*100}%")
#print(f"The mean test accuracy obtained: {mean_acc*100}%")
print(f"The ensemble test accuracy obtained: {ensemble_accuracy*100}%")

The best model 3 test accuracy obtained: 89.13043737411499%
The ensemble test accuracy obtained: 19.848771266540645%


# Summary/Discussion

I started by using three models in an ensemble using majority voting, unfortunately the training for the three models takes over 10 hours and failed due to memory issues. Instead I decided to use only the 3rd model as you can see in the code all the code for the 1st and 2nd models has been commented out. The number of epochs used initially was 60, which resulted in accuracy starting around 45% but took far too long to train. I used the MAE history graph and the validation accuracy and loss graphs to determine the range of epochs I needed to use, which ended up around 10 resulting in 80%+ accuracy. The graphs for training and validation accuracy and loss can be seen above, as well as the mean test accuracy of 89.13%. Initially I had used another pre-trained network called VGG19 but the total training for all three pre-trained models was too extensive as well, to mitigate training for too long I created a custom cnn model. As well I had to try different batch sizes, data augmentation, and target sizes to get optimal results that didn't take hours to train.